In [2]:
import numpy as np
import random
from scipy.stats import norm

In [3]:
def GGM_instance(n=100, p=100, max_edges=10):
    def generate_vertices(p):
        vertices = np.random.uniform(size=(p,2))
        return vertices
    def connecting_prob(v1,v2,p):
        # Euclidean distance of v1, v2
        d = np.linalg.norm(v1-v2)
        # calculating connecting probability
        prob = norm.pdf(d/np.sqrt(p))
        return prob
    def remove_edges(p, adj, max_edges):
        idx = list(range(p))
        np.random.shuffle(idx)

        for i in idx:
            if np.all(np.sum(adj, axis=1) <= (max_edges+1)):
                break
            # Indices of nodes connected to v_i
            nonzero_i = list(np.nonzero(adj[i])[0])
            n_edges = len(nonzero_i)

            # Delete some edges if there are redundancies
            if n_edges > (max_edges+1):
                nonzero_i.remove(i)
                removed_idx_i = random.sample(nonzero_i,n_edges-max_edges)
                # Remove other edges
                adj[i,removed_idx_i] = 0
                adj[removed_idx_i,i] = 0

        return adj

    vertices = generate_vertices(p)

    adj_mat = np.eye(p)

    for i in range(p):
        for j in range(i+1,p):
            v_i = vertices[i]
            v_j = vertices[j]
            adj_mat[i,j] = np.random.binomial(n=1,
                                              p=connecting_prob(v1=v_i,
                                                                v2=v_j,
                                                                p=p))

    # symmetrize
    adj_mat = adj_mat + adj_mat.T - np.eye(p)

    # remove redundant edges
    adj_mat = remove_edges(p, adj_mat, max_edges)

    # maximal off-diag value to guarantee diagonal dominance
    max_off_diag = 1/max_edges
    max_off_diag = max_off_diag*0.9

    # generate a PD precision
    precision = np.random.uniform(low=-max_off_diag,high=max_off_diag,
                                  size=(p,p))
    # symmetrize precision
    precision = np.tril(precision)
    precision = precision + precision.T
    # sparsify precision based on adjacency matrix
    precision = precision * adj_mat
    np.fill_diagonal(precision, 1)
    cov = np.linalg.inv(precision)

    # standardize the covariance
    cov = cov/np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))
    precision = np.linalg.inv(cov)

    X = np.random.multivariate_normal(mean=np.zeros(p),
                                      cov=cov, size=n)

    return precision, cov, X

In [4]:
prec,cov,X = GGM_instance(p=10, max_edges=3)

In [8]:
np.set_printoptions(precision=4, suppress=True)
print(cov)

[[ 1.      0.0004 -0.0001  0.0003 -0.008   0.1313  0.0265 -0.0054  0.002
   0.0168]
 [ 0.0004  1.      0.0006 -0.0035  0.0465 -0.0059  0.1533  0.0676 -0.0227
  -0.2103]
 [-0.0001  0.0006  1.     -0.1688  0.0012 -0.0002  0.0004  0.0081 -0.0261
  -0.0026]
 [ 0.0003 -0.0035 -0.1688  1.     -0.007   0.0009 -0.0025 -0.0477  0.1548
   0.0153]
 [-0.008   0.0465  0.0012 -0.007   1.     -0.1296  0.298   0.0168 -0.0453
  -0.0141]
 [ 0.1313 -0.0059 -0.0002  0.0009 -0.1296  1.     -0.0349 -0.0029  0.0061
   0.004 ]
 [ 0.0265  0.1533  0.0004 -0.0025  0.298  -0.0349  1.      0.0142 -0.0163
  -0.033 ]
 [-0.0054  0.0676  0.0081 -0.0477  0.0168 -0.0029  0.0142  1.     -0.3084
  -0.319 ]
 [ 0.002  -0.0227 -0.0261  0.1548 -0.0453  0.0061 -0.0163 -0.3084  1.
   0.0987]
 [ 0.0168 -0.2103 -0.0026  0.0153 -0.0141  0.004  -0.033  -0.319   0.0987
   1.    ]]


In [9]:
print(prec)

[[ 1.0189 -0.     -0.     -0.     -0.     -0.1349 -0.0323  0.      0.
  -0.0176]
 [ 0.      1.0703 -0.     -0.     -0.     -0.     -0.1568 -0.      0.
   0.2199]
 [ 0.      0.      1.0293  0.1738  0.      0.     -0.      0.      0.
  -0.    ]
 [ 0.     -0.      0.1738  1.0539  0.      0.     -0.     -0.     -0.1586
  -0.    ]
 [-0.     -0.      0.      0.      1.1168  0.1331 -0.3275 -0.      0.0445
  -0.    ]
 [-0.1349 -0.      0.      0.      0.1331  1.035   0.     -0.     -0.
  -0.    ]
 [-0.0323 -0.1568  0.     -0.     -0.3275  0.      1.1225  0.      0.
   0.    ]
 [ 0.      0.      0.     -0.      0.     -0.      0.      1.2182  0.3406
   0.3549]
 [-0.      0.     -0.     -0.1586  0.0445 -0.      0.      0.3406  1.1316
   0.    ]
 [-0.0176  0.2199  0.     -0.      0.     -0.      0.      0.3549  0.
   1.1598]]
